In [1]:
import os, sys, pathlib, importlib, glob

# 1) Point to repo root and src/
ROOT = pathlib.Path.cwd()
if not (ROOT / "src" / "micro_lm").exists():
    for _ in range(6):
        ROOT = ROOT.parent
        if (ROOT / "src" / "micro_lm").exists():
            break
SRC = ROOT / "src"
print("Repo root:", ROOT)
print("Using src:", SRC)

# 2) Ensure src/ is first on sys.path
if str(SRC) in sys.path:
    sys.path.remove(str(SRC))
sys.path.insert(0, str(SRC))

# 3) Make sure packages are *real* packages (create __init__.py if missing)
PKG = SRC / "micro_lm"
CLI = PKG / "cli"
for p in (PKG, CLI):
    if p.exists() and not (p / "__init__.py").exists():
        (p / "__init__.py").write_text("")  # minimal package
        print("Created:", p / "__init__.py")

# 4) Purge any previously-imported installed micro_lm modules
for k in list(sys.modules.keys()):
    if k == "micro_lm" or k.startswith("micro_lm."):
        del sys.modules[k]
importlib.invalidate_caches()

# 5) Import micro_lm *from src* and then the CLI module
import micro_lm
print("micro_lm loaded from:", micro_lm.__file__)

from micro_lm.cli.defi_quickstart import quickstart
print("quickstart loaded OK from src.")


Repo root: /Users/ian_moore/repos/micro-lm
Using src: /Users/ian_moore/repos/micro-lm/src
micro_lm loaded from: /Users/ian_moore/repos/micro-lm/src/micro_lm/__init__.py
quickstart loaded OK from src.


In [2]:
import os, json

# Show WDD debug lines (same as setting MICRO_LM_WDD_DEBUG=1 on the CLI)
os.environ["MICRO_LM_WDD_DEBUG"] = "1"

from micro_lm.cli.defi_quickstart import quickstart

prompt = "deposit 10 ETH into aave"
policy = {"audit": {"backend": "wdd"}}   # same as --policy '{"audit":{"backend":"wdd"}}'
rails  = "stage11"

# If your build requires the explicit switch for detector mode, set use_wdd=True.
# Otherwise you can omit it.
out = quickstart(
    prompt,
    policy=policy,
    rails=rails,
    T=180,
    use_wdd=True,       # <- set to True if WDD needs to be explicitly invoked
    verbose=True,       # matches --verbose
)

print(json.dumps(out, indent=2))


[WDD] prompt='deposit 10 ETH into aave' | seq=['deposit_asset']
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
[WDD] act=deposit layer=-5 sigma=4 proto_w=13 mf_peak=6.953530481900707 keep_len=0 decision=PASS
[WDD] fallback: MF_peak=6.95


{
  "prompt": "deposit 10 ETH into aave",
  "domain": "defi",
  "rails": "stage11",
  "T": 180,
  "top1": "deposit_asset",
  "sequence": [
    "deposit_asset"
  ],
  "plan": {
    "sequence": [
      "deposit_asset"
    ]
  },
  "verify": {
    "ok": true,
    "reason": "shim:accept:stage-4",
    "tags": [
      "rails:stage11",
      "wdd:on",
      "audit:wdd"
    ]
  },
  "flags": {},
  "aux": {
    "stage11": {
      "wdd": {
        "decision": "PASS",
        "sigma": 4,
        "proto_w": 13,
        "which_prior": "deposit(L-5)",
        "mf_peak": 6.953530481900707,
        "keep": []
      }
    }
  },
  "det_hash": "f1378c645f25",
  "wdd_summary": {
    "decision": "PASS",
    "keep": [],
    "sigma": 4,
    "proto_w": 13,
    "which_prior": "deposit(L-5)",
    "note": "fallback: MF_peak=6.953530481900707"
  },
  "abstained": false
}
